# FSIP2 Mutation Trans Effect on Phosphoproteomics

This notebook analyzes the trans effect of FSIP2 mutation on interacting and other proteins Phosphoproteomics, in Endometrial, Colon, and Ovarian cancer.

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import collections
import re
import gseapy as gp
from gseapy.plot import barplot, dotplot
import matplotlib.pyplot as plt
import seaborn as sns

import cptac
import cptac.algorithms as al

en = cptac.Endometrial()
co = cptac.Colon()
ov = cptac.Ovarian()

### Select Gene

In [2]:
gene = "FSIP2"

### Investigate Proteomics, Phosphoproteomics, Acetylproteomics, or Transcriptomics

In [3]:
#omics = "proteomics"
#omics = "transcriptomics"
omics = "phosphoproteomics"
#omics = "acetylproteomics"

# Interacting Proteins: Transcriptomics

## Endometrial

### Generate interacting protein list

In [4]:
'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

omics_object = en.get_phosphoproteomics()

print("Generating interacting protein list")
interacting_proteins_in_omics_df = []

'''Only do comparisons on proteins in the omics dataframe'''
for ip in interacting_proteins:
    if omics == 'phosphoproteomics' or omics == 'acetylproteomics':
        col_regex = ip + "-.*" # Build a regex to get all columns that match the gene
    else:
        col_regex = '^{}$'.format(ip)

    selected = omics_object.filter(regex=col_regex)

    if len(selected.columns) > 0:
        interacting_proteins_in_omics_df.append(ip)

print("Interacting Proteins:")
for interacting_protein in interacting_proteins_in_omics_df:
    print(interacting_protein)

Generating interacting protein list
Interacting Proteins:
AKAP3
FSIP1
PDZRN3
SNTB1
ABCA13
ZNF445
SPATS2


### Test for significant comparisons in any of interacting proteins

In [5]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = en.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins_in_omics_df)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)


'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")

Doing t-test comparisons



/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3157: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


No significant comparisons.


## Colon

### Generate interacting protein list

In [6]:
'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

omics_object = co.get_phosphoproteomics()

print("Generating interacting protein list")
interacting_proteins_in_omics_df = []

'''Only do comparisons on proteins in the omics dataframe'''
for ip in interacting_proteins:
    if omics == 'phosphoproteomics' or omics == 'acetylproteomics':
        col_regex = ip + "_.*" # Build a regex to get all columns that match the gene
    else:
        col_regex = '^{}$'.format(ip)

    selected = omics_object.filter(regex=col_regex)

    if len(selected.columns) > 0:
        interacting_proteins_in_omics_df.append(ip)

print("Interacting Proteins:")
for interacting_protein in interacting_proteins_in_omics_df:
    print(interacting_protein)

Generating interacting protein list
Interacting Proteins:
PDZRN3
SNTB1
POTEI
ZNF445
SPATS2


### Test for significant comparisons in any of interacting proteins

In [7]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = co.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins_in_omics_df)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)

'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")

PDZRN3 did not match any columns in phosphoproteomics dataframe. PDZRN3_phosphoproteomics column inserted, but filled with NaN.
SNTB1 did not match any columns in phosphoproteomics dataframe. SNTB1_phosphoproteomics column inserted, but filled with NaN.
POTEI did not match any columns in phosphoproteomics dataframe. POTEI_phosphoproteomics column inserted, but filled with NaN.
ZNF445 did not match any columns in phosphoproteomics dataframe. ZNF445_phosphoproteomics column inserted, but filled with NaN.
SPATS2 did not match any columns in phosphoproteomics dataframe. SPATS2_phosphoproteomics column inserted, but filled with NaN.
Doing t-test comparisons

No significant comparisons.


## Ovarian

### Generate interacting protein list

In [8]:
'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

omics_object = ov.get_phosphoproteomics()

print("Generating interacting protein list")
interacting_proteins_in_omics_df = []

'''Only do comparisons on proteins in the omics dataframe'''
for ip in interacting_proteins:
    if omics == 'phosphoproteomics' or omics == 'acetylproteomics':
        col_regex = ip + "-.*" # Build a regex to get all columns that match the gene
    else:
        col_regex = '^{}$'.format(ip)

    selected = omics_object.filter(regex=col_regex)

    if len(selected.columns) > 0:
        interacting_proteins_in_omics_df.append(ip)

print("Interacting Proteins:")
for interacting_protein in interacting_proteins_in_omics_df:
    print(interacting_protein)

Generating interacting protein list
Interacting Proteins:
AKAP3
CMBL
PDZRN3
SNTB1
POTEI
ABCA13
ZNF445
SPATS2


### Test for Significant Comparisons

In [9]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = ov.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins_in_omics_df) 
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.loc[ind,'Label'] = 'Mutated'
    else:
        protdf.loc[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)
protdf = protdf.loc[:,~protdf.columns.duplicated()]

'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")

Doing t-test comparisons



/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3157: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


No significant comparisons.


# All Proteins: Phosphoproteomics

## Endometrial

In [10]:
try:
    print("\nGene: ", gene)

    '''Use all proteins'''
    proteomics = en.get_proteomics()
    all_proteins = proteomics.columns
    #all_proteins = all_proteins[:100]

    '''Create dataframe in order to do comparisons with wrap_ttest'''
    protdf = en.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=all_proteins)
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    '''Create the binary valued column needed to do the comparison'''
    for ind, row in protdf.iterrows():
        if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
            protdf.at[ind,'Label'] = 'Mutated'
        else:
            protdf.at[ind,'Label'] = 'Wildtype'

    '''Format the datafram correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    '''Make list of columns to be compared using t-tests'''
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    '''Call wrap_ttest, pass in formatted dataframe'''
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    '''Print results, if anything significant was found'''
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")

except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  FSIP2
A1BG did not match any columns in phosphoproteomics dataframe. A1BG_phosphoproteomics column inserted, but filled with NaN.
A2M did not match any columns in phosphoproteomics dataframe. A2M_phosphoproteomics column inserted, but filled with NaN.
A2ML1 did not match any columns in phosphoproteomics dataframe. A2ML1_phosphoproteomics column inserted, but filled with NaN.
A4GALT did not match any columns in phosphoproteomics dataframe. A4GALT_phosphoproteomics column inserted, but filled with NaN.
AADAT did not match any columns in phosphoproteomics dataframe. AADAT_phosphoproteomics column inserted, but filled with NaN.
AAMP did not match any columns in phosphoproteomics dataframe. AAMP_phosphoproteomics column inserted, but filled with NaN.
AAR2 did not match any columns in phosphoproteomics dataframe. AAR2_phosphoproteomics column inserted, but filled with NaN.
AARS2 did not match any columns in phosphoproteomics dataframe. AARS2_phosphoproteomics column inserted, but fil

ACSF3 did not match any columns in phosphoproteomics dataframe. ACSF3_phosphoproteomics column inserted, but filled with NaN.
ACSL1 did not match any columns in phosphoproteomics dataframe. ACSL1_phosphoproteomics column inserted, but filled with NaN.
ACSM3 did not match any columns in phosphoproteomics dataframe. ACSM3_phosphoproteomics column inserted, but filled with NaN.
ACSS1 did not match any columns in phosphoproteomics dataframe. ACSS1_phosphoproteomics column inserted, but filled with NaN.
ACTA1 did not match any columns in phosphoproteomics dataframe. ACTA1_phosphoproteomics column inserted, but filled with NaN.
ACTA2 did not match any columns in phosphoproteomics dataframe. ACTA2_phosphoproteomics column inserted, but filled with NaN.
ACTB did not match any columns in phosphoproteomics dataframe. ACTB_phosphoproteomics column inserted, but filled with NaN.
ACTG1 did not match any columns in phosphoproteomics dataframe. ACTG1_phosphoproteomics column inserted, but filled with

AGPAT3 did not match any columns in phosphoproteomics dataframe. AGPAT3_phosphoproteomics column inserted, but filled with NaN.
AGPAT5 did not match any columns in phosphoproteomics dataframe. AGPAT5_phosphoproteomics column inserted, but filled with NaN.
AGR2 did not match any columns in phosphoproteomics dataframe. AGR2_phosphoproteomics column inserted, but filled with NaN.
AGR3 did not match any columns in phosphoproteomics dataframe. AGR3_phosphoproteomics column inserted, but filled with NaN.
AGT did not match any columns in phosphoproteomics dataframe. AGT_phosphoproteomics column inserted, but filled with NaN.
AHCY did not match any columns in phosphoproteomics dataframe. AHCY_phosphoproteomics column inserted, but filled with NaN.
AHSP did not match any columns in phosphoproteomics dataframe. AHSP_phosphoproteomics column inserted, but filled with NaN.
AIFM2 did not match any columns in phosphoproteomics dataframe. AIFM2_phosphoproteomics column inserted, but filled with NaN.


ANG did not match any columns in phosphoproteomics dataframe. ANG_phosphoproteomics column inserted, but filled with NaN.
ANGEL2 did not match any columns in phosphoproteomics dataframe. ANGEL2_phosphoproteomics column inserted, but filled with NaN.
ANGPT2 did not match any columns in phosphoproteomics dataframe. ANGPT2_phosphoproteomics column inserted, but filled with NaN.
ANGPT4 did not match any columns in phosphoproteomics dataframe. ANGPT4_phosphoproteomics column inserted, but filled with NaN.
ANGPTL1 did not match any columns in phosphoproteomics dataframe. ANGPTL1_phosphoproteomics column inserted, but filled with NaN.
ANGPTL4 did not match any columns in phosphoproteomics dataframe. ANGPTL4_phosphoproteomics column inserted, but filled with NaN.
ANKHD1 did not match any columns in phosphoproteomics dataframe. ANKHD1_phosphoproteomics column inserted, but filled with NaN.
ANKRD16 did not match any columns in phosphoproteomics dataframe. ANKRD16_phosphoproteomics column inserte

ARF1 did not match any columns in phosphoproteomics dataframe. ARF1_phosphoproteomics column inserted, but filled with NaN.
ARF3 did not match any columns in phosphoproteomics dataframe. ARF3_phosphoproteomics column inserted, but filled with NaN.
ARFRP1 did not match any columns in phosphoproteomics dataframe. ARFRP1_phosphoproteomics column inserted, but filled with NaN.
ARG2 did not match any columns in phosphoproteomics dataframe. ARG2_phosphoproteomics column inserted, but filled with NaN.
ARIH2 did not match any columns in phosphoproteomics dataframe. ARIH2_phosphoproteomics column inserted, but filled with NaN.
ARL1 did not match any columns in phosphoproteomics dataframe. ARL1_phosphoproteomics column inserted, but filled with NaN.
ARL15 did not match any columns in phosphoproteomics dataframe. ARL15_phosphoproteomics column inserted, but filled with NaN.
ARL4A did not match any columns in phosphoproteomics dataframe. ARL4A_phosphoproteomics column inserted, but filled with NaN

...

UBAC2 did not match any columns in phosphoproteomics dataframe. UBAC2_phosphoproteomics column inserted, but filled with NaN.
UBALD1 did not match any columns in phosphoproteomics dataframe. UBALD1_phosphoproteomics column inserted, but filled with NaN.
UBALD2 did not match any columns in phosphoproteomics dataframe. UBALD2_phosphoproteomics column inserted, but filled with NaN.
UBB did not match any columns in phosphoproteomics dataframe. UBB_phosphoproteomics column inserted, but filled with NaN.
UBE2A did not match any columns in phosphoproteomics dataframe. UBE2A_phosphoproteomics column inserted, but filled with NaN.
UBE2B did not match any columns in phosphoproteomics dataframe. UBE2B_phosphoproteomics column inserted, but filled with NaN.
UBE2C did not match any columns in phosphoproteomics dataframe. UBE2C_phosphoproteomics column inserted, but filled with NaN.
UBE2D1 did not match any columns in phosphoproteomics dataframe. UBE2D1_phosphoproteomics column inserted, but filled 

UTP4 did not match any columns in phosphoproteomics dataframe. UTP4_phosphoproteomics column inserted, but filled with NaN.
UTP6 did not match any columns in phosphoproteomics dataframe. UTP6_phosphoproteomics column inserted, but filled with NaN.
UXS1 did not match any columns in phosphoproteomics dataframe. UXS1_phosphoproteomics column inserted, but filled with NaN.
UXT did not match any columns in phosphoproteomics dataframe. UXT_phosphoproteomics column inserted, but filled with NaN.
VAMP2 did not match any columns in phosphoproteomics dataframe. VAMP2_phosphoproteomics column inserted, but filled with NaN.
VAMP7 did not match any columns in phosphoproteomics dataframe. VAMP7_phosphoproteomics column inserted, but filled with NaN.
VARS2 did not match any columns in phosphoproteomics dataframe. VARS2_phosphoproteomics column inserted, but filled with NaN.
VBP1 did not match any columns in phosphoproteomics dataframe. VBP1_phosphoproteomics column inserted, but filled with NaN.
VCAM

WNT7A did not match any columns in phosphoproteomics dataframe. WNT7A_phosphoproteomics column inserted, but filled with NaN.
WRB did not match any columns in phosphoproteomics dataframe. WRB_phosphoproteomics column inserted, but filled with NaN.
WRB-SH3BGR did not match any columns in phosphoproteomics dataframe. WRB-SH3BGR_phosphoproteomics column inserted, but filled with NaN.
XAGE1A did not match any columns in phosphoproteomics dataframe. XAGE1A_phosphoproteomics column inserted, but filled with NaN.
XCL1 did not match any columns in phosphoproteomics dataframe. XCL1_phosphoproteomics column inserted, but filled with NaN.
XDH did not match any columns in phosphoproteomics dataframe. XDH_phosphoproteomics column inserted, but filled with NaN.
XG did not match any columns in phosphoproteomics dataframe. XG_phosphoproteomics column inserted, but filled with NaN.
XPNPEP3 did not match any columns in phosphoproteomics dataframe. XPNPEP3_phosphoproteomics column inserted, but filled wi

ZNF302 did not match any columns in phosphoproteomics dataframe. ZNF302_phosphoproteomics column inserted, but filled with NaN.
ZNF304 did not match any columns in phosphoproteomics dataframe. ZNF304_phosphoproteomics column inserted, but filled with NaN.
ZNF32 did not match any columns in phosphoproteomics dataframe. ZNF32_phosphoproteomics column inserted, but filled with NaN.
ZNF329 did not match any columns in phosphoproteomics dataframe. ZNF329_phosphoproteomics column inserted, but filled with NaN.
ZNF33A did not match any columns in phosphoproteomics dataframe. ZNF33A_phosphoproteomics column inserted, but filled with NaN.
ZNF346 did not match any columns in phosphoproteomics dataframe. ZNF346_phosphoproteomics column inserted, but filled with NaN.
ZNF37A did not match any columns in phosphoproteomics dataframe. ZNF37A_phosphoproteomics column inserted, but filled with NaN.
ZNF41 did not match any columns in phosphoproteomics dataframe. ZNF41_phosphoproteomics column inserted, b

ZWINT did not match any columns in phosphoproteomics dataframe. ZWINT_phosphoproteomics column inserted, but filled with NaN.
ZYG11B did not match any columns in phosphoproteomics dataframe. ZYG11B_phosphoproteomics column inserted, but filled with NaN.
Doing t-test comparisons

No significant comparisons.


## Colon

In [11]:
try:
    print("\nGene: ", gene)

    '''Use all proteins'''
    proteomics = co.get_proteomics()
    all_proteins = proteomics.columns
    #all_proteins = all_proteins[:100]

    '''Create dataframe in order to do comparisons with wrap_ttest'''
    protdf = co.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=all_proteins)
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    '''Create the binary valued column needed to do the comparison'''
    for ind, row in protdf.iterrows():
        if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
            protdf.at[ind,'Label'] = 'Mutated'
        else:
            protdf.at[ind,'Label'] = 'Wildtype'

    '''Format the datafram correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    '''Make list of columns to be compared using t-tests'''
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    '''Call wrap_ttest, pass in formatted dataframe'''
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    '''Print results, if anything significant was found'''
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")

except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  FSIP2
A1BG did not match any columns in phosphoproteomics dataframe. A1BG_phosphoproteomics column inserted, but filled with NaN.
A1CF did not match any columns in phosphoproteomics dataframe. A1CF_phosphoproteomics column inserted, but filled with NaN.
A2M did not match any columns in phosphoproteomics dataframe. A2M_phosphoproteomics column inserted, but filled with NaN.
AAAS did not match any columns in phosphoproteomics dataframe. AAAS_phosphoproteomics column inserted, but filled with NaN.
AACS did not match any columns in phosphoproteomics dataframe. AACS_phosphoproteomics column inserted, but filled with NaN.
AAGAB did not match any columns in phosphoproteomics dataframe. AAGAB_phosphoproteomics column inserted, but filled with NaN.
AAK1 did not match any columns in phosphoproteomics dataframe. AAK1_phosphoproteomics column inserted, but filled with NaN.
AAMDC did not match any columns in phosphoproteomics dataframe. AAMDC_phosphoproteomics column inserted, but filled wi

ACAP2 did not match any columns in phosphoproteomics dataframe. ACAP2_phosphoproteomics column inserted, but filled with NaN.
ACAT1 did not match any columns in phosphoproteomics dataframe. ACAT1_phosphoproteomics column inserted, but filled with NaN.
ACAT2 did not match any columns in phosphoproteomics dataframe. ACAT2_phosphoproteomics column inserted, but filled with NaN.
ACBD3 did not match any columns in phosphoproteomics dataframe. ACBD3_phosphoproteomics column inserted, but filled with NaN.
ACBD5 did not match any columns in phosphoproteomics dataframe. ACBD5_phosphoproteomics column inserted, but filled with NaN.
ACBD6 did not match any columns in phosphoproteomics dataframe. ACBD6_phosphoproteomics column inserted, but filled with NaN.
ACE did not match any columns in phosphoproteomics dataframe. ACE_phosphoproteomics column inserted, but filled with NaN.
ACE2 did not match any columns in phosphoproteomics dataframe. ACE2_phosphoproteomics column inserted, but filled with NaN

ADAM9 did not match any columns in phosphoproteomics dataframe. ADAM9_phosphoproteomics column inserted, but filled with NaN.
ADAMDEC1 did not match any columns in phosphoproteomics dataframe. ADAMDEC1_phosphoproteomics column inserted, but filled with NaN.
ADAMTS16 did not match any columns in phosphoproteomics dataframe. ADAMTS16_phosphoproteomics column inserted, but filled with NaN.
ADAMTSL4 did not match any columns in phosphoproteomics dataframe. ADAMTSL4_phosphoproteomics column inserted, but filled with NaN.
ADAP1 did not match any columns in phosphoproteomics dataframe. ADAP1_phosphoproteomics column inserted, but filled with NaN.
ADAR did not match any columns in phosphoproteomics dataframe. ADAR_phosphoproteomics column inserted, but filled with NaN.
ADCK5 did not match any columns in phosphoproteomics dataframe. ADCK5_phosphoproteomics column inserted, but filled with NaN.
ADCY6 did not match any columns in phosphoproteomics dataframe. ADCY6_phosphoproteomics column inserte

AGTRAP did not match any columns in phosphoproteomics dataframe. AGTRAP_phosphoproteomics column inserted, but filled with NaN.
AHCTF1 did not match any columns in phosphoproteomics dataframe. AHCTF1_phosphoproteomics column inserted, but filled with NaN.
AHCY did not match any columns in phosphoproteomics dataframe. AHCY_phosphoproteomics column inserted, but filled with NaN.
AHCYL1 did not match any columns in phosphoproteomics dataframe. AHCYL1_phosphoproteomics column inserted, but filled with NaN.
AHCYL2 did not match any columns in phosphoproteomics dataframe. AHCYL2_phosphoproteomics column inserted, but filled with NaN.
AHNAK did not match any columns in phosphoproteomics dataframe. AHNAK_phosphoproteomics column inserted, but filled with NaN.
AHNAK2 did not match any columns in phosphoproteomics dataframe. AHNAK2_phosphoproteomics column inserted, but filled with NaN.
AHR did not match any columns in phosphoproteomics dataframe. AHR_phosphoproteomics column inserted, but fille

ALDH3A1 did not match any columns in phosphoproteomics dataframe. ALDH3A1_phosphoproteomics column inserted, but filled with NaN.
ALDH3A2 did not match any columns in phosphoproteomics dataframe. ALDH3A2_phosphoproteomics column inserted, but filled with NaN.
ALDH3B1 did not match any columns in phosphoproteomics dataframe. ALDH3B1_phosphoproteomics column inserted, but filled with NaN.
ALDH4A1 did not match any columns in phosphoproteomics dataframe. ALDH4A1_phosphoproteomics column inserted, but filled with NaN.
ALDH5A1 did not match any columns in phosphoproteomics dataframe. ALDH5A1_phosphoproteomics column inserted, but filled with NaN.
ALDH6A1 did not match any columns in phosphoproteomics dataframe. ALDH6A1_phosphoproteomics column inserted, but filled with NaN.
ALDH7A1 did not match any columns in phosphoproteomics dataframe. ALDH7A1_phosphoproteomics column inserted, but filled with NaN.
ALDH9A1 did not match any columns in phosphoproteomics dataframe. ALDH9A1_phosphoproteomic

...

VPS52 did not match any columns in phosphoproteomics dataframe. VPS52_phosphoproteomics column inserted, but filled with NaN.
VPS53 did not match any columns in phosphoproteomics dataframe. VPS53_phosphoproteomics column inserted, but filled with NaN.
VPS72 did not match any columns in phosphoproteomics dataframe. VPS72_phosphoproteomics column inserted, but filled with NaN.
VPS8 did not match any columns in phosphoproteomics dataframe. VPS8_phosphoproteomics column inserted, but filled with NaN.
VRK1 did not match any columns in phosphoproteomics dataframe. VRK1_phosphoproteomics column inserted, but filled with NaN.
VRK2 did not match any columns in phosphoproteomics dataframe. VRK2_phosphoproteomics column inserted, but filled with NaN.
VRK3 did not match any columns in phosphoproteomics dataframe. VRK3_phosphoproteomics column inserted, but filled with NaN.
VSIG2 did not match any columns in phosphoproteomics dataframe. VSIG2_phosphoproteomics column inserted, but filled with NaN.


WDR73 did not match any columns in phosphoproteomics dataframe. WDR73_phosphoproteomics column inserted, but filled with NaN.
WDR74 did not match any columns in phosphoproteomics dataframe. WDR74_phosphoproteomics column inserted, but filled with NaN.
WDR75 did not match any columns in phosphoproteomics dataframe. WDR75_phosphoproteomics column inserted, but filled with NaN.
WDR77 did not match any columns in phosphoproteomics dataframe. WDR77_phosphoproteomics column inserted, but filled with NaN.
WDR81 did not match any columns in phosphoproteomics dataframe. WDR81_phosphoproteomics column inserted, but filled with NaN.
WDR82 did not match any columns in phosphoproteomics dataframe. WDR82_phosphoproteomics column inserted, but filled with NaN.
WDR83 did not match any columns in phosphoproteomics dataframe. WDR83_phosphoproteomics column inserted, but filled with NaN.
WDR89 did not match any columns in phosphoproteomics dataframe. WDR89_phosphoproteomics column inserted, but filled wi

YBX1 did not match any columns in phosphoproteomics dataframe. YBX1_phosphoproteomics column inserted, but filled with NaN.
YBX2 did not match any columns in phosphoproteomics dataframe. YBX2_phosphoproteomics column inserted, but filled with NaN.
YBX3 did not match any columns in phosphoproteomics dataframe. YBX3_phosphoproteomics column inserted, but filled with NaN.
YEATS4 did not match any columns in phosphoproteomics dataframe. YEATS4_phosphoproteomics column inserted, but filled with NaN.
YES1 did not match any columns in phosphoproteomics dataframe. YES1_phosphoproteomics column inserted, but filled with NaN.
YIF1A did not match any columns in phosphoproteomics dataframe. YIF1A_phosphoproteomics column inserted, but filled with NaN.
YIF1B did not match any columns in phosphoproteomics dataframe. YIF1B_phosphoproteomics column inserted, but filled with NaN.
YIPF3 did not match any columns in phosphoproteomics dataframe. YIPF3_phosphoproteomics column inserted, but filled with NaN

ZFAND1 did not match any columns in phosphoproteomics dataframe. ZFAND1_phosphoproteomics column inserted, but filled with NaN.
ZFAND2B did not match any columns in phosphoproteomics dataframe. ZFAND2B_phosphoproteomics column inserted, but filled with NaN.
ZFAND5 did not match any columns in phosphoproteomics dataframe. ZFAND5_phosphoproteomics column inserted, but filled with NaN.
ZFAND6 did not match any columns in phosphoproteomics dataframe. ZFAND6_phosphoproteomics column inserted, but filled with NaN.
ZFP36 did not match any columns in phosphoproteomics dataframe. ZFP36_phosphoproteomics column inserted, but filled with NaN.
ZFP36L1 did not match any columns in phosphoproteomics dataframe. ZFP36L1_phosphoproteomics column inserted, but filled with NaN.
ZFP36L2 did not match any columns in phosphoproteomics dataframe. ZFP36L2_phosphoproteomics column inserted, but filled with NaN.
ZFP62 did not match any columns in phosphoproteomics dataframe. ZFP62_phosphoproteomics column inser

ZNF627 did not match any columns in phosphoproteomics dataframe. ZNF627_phosphoproteomics column inserted, but filled with NaN.
ZNF638 did not match any columns in phosphoproteomics dataframe. ZNF638_phosphoproteomics column inserted, but filled with NaN.
ZNF662 did not match any columns in phosphoproteomics dataframe. ZNF662_phosphoproteomics column inserted, but filled with NaN.
ZNF687 did not match any columns in phosphoproteomics dataframe. ZNF687_phosphoproteomics column inserted, but filled with NaN.
ZNF703 did not match any columns in phosphoproteomics dataframe. ZNF703_phosphoproteomics column inserted, but filled with NaN.
ZNF705E did not match any columns in phosphoproteomics dataframe. ZNF705E_phosphoproteomics column inserted, but filled with NaN.
ZNF706 did not match any columns in phosphoproteomics dataframe. ZNF706_phosphoproteomics column inserted, but filled with NaN.
ZNF74 did not match any columns in phosphoproteomics dataframe. ZNF74_phosphoproteomics column inserte

## Ovarian

In [12]:

print("\nGene: ", gene)

'''Use all proteins'''
proteomics = ov.get_proteomics()
all_proteins = list(set(proteomics.columns))

'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = ov.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=all_proteins)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
protdf = protdf.loc[:,~protdf.columns.duplicated()]

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the datafram correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)

'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")



Gene:  FSIP2
ELN did not match any columns in phosphoproteomics dataframe. ELN_phosphoproteomics column inserted, but filled with NaN.
RAP2C did not match any columns in phosphoproteomics dataframe. RAP2C_phosphoproteomics column inserted, but filled with NaN.
ZDHHC24 did not match any columns in phosphoproteomics dataframe. ZDHHC24_phosphoproteomics column inserted, but filled with NaN.
NNT did not match any columns in phosphoproteomics dataframe. NNT_phosphoproteomics column inserted, but filled with NaN.
STARD5 did not match any columns in phosphoproteomics dataframe. STARD5_phosphoproteomics column inserted, but filled with NaN.
COX6A1 did not match any columns in phosphoproteomics dataframe. COX6A1_phosphoproteomics column inserted, but filled with NaN.
TNPO3 did not match any columns in phosphoproteomics dataframe. TNPO3_phosphoproteomics column inserted, but filled with NaN.
CACUL1 did not match any columns in phosphoproteomics dataframe. CACUL1_phosphoproteomics column inserte

AQP5 did not match any columns in phosphoproteomics dataframe. AQP5_phosphoproteomics column inserted, but filled with NaN.
GPAM did not match any columns in phosphoproteomics dataframe. GPAM_phosphoproteomics column inserted, but filled with NaN.
LAMP2 did not match any columns in phosphoproteomics dataframe. LAMP2_phosphoproteomics column inserted, but filled with NaN.
STX8 did not match any columns in phosphoproteomics dataframe. STX8_phosphoproteomics column inserted, but filled with NaN.
LDHAL6B did not match any columns in phosphoproteomics dataframe. LDHAL6B_phosphoproteomics column inserted, but filled with NaN.
GPC4 did not match any columns in phosphoproteomics dataframe. GPC4_phosphoproteomics column inserted, but filled with NaN.
ENTPD1 did not match any columns in phosphoproteomics dataframe. ENTPD1_phosphoproteomics column inserted, but filled with NaN.
PCSK5 did not match any columns in phosphoproteomics dataframe. PCSK5_phosphoproteomics column inserted, but filled with

SELENOT did not match any columns in phosphoproteomics dataframe. SELENOT_phosphoproteomics column inserted, but filled with NaN.
GBP4 did not match any columns in phosphoproteomics dataframe. GBP4_phosphoproteomics column inserted, but filled with NaN.
PORCN did not match any columns in phosphoproteomics dataframe. PORCN_phosphoproteomics column inserted, but filled with NaN.
CBLN1 did not match any columns in phosphoproteomics dataframe. CBLN1_phosphoproteomics column inserted, but filled with NaN.
PFN2 did not match any columns in phosphoproteomics dataframe. PFN2_phosphoproteomics column inserted, but filled with NaN.
CEP120 did not match any columns in phosphoproteomics dataframe. CEP120_phosphoproteomics column inserted, but filled with NaN.
TMEM120A did not match any columns in phosphoproteomics dataframe. TMEM120A_phosphoproteomics column inserted, but filled with NaN.
UBE2K did not match any columns in phosphoproteomics dataframe. UBE2K_phosphoproteomics column inserted, but f

ZNF567 did not match any columns in phosphoproteomics dataframe. ZNF567_phosphoproteomics column inserted, but filled with NaN.
GUK1 did not match any columns in phosphoproteomics dataframe. GUK1_phosphoproteomics column inserted, but filled with NaN.
PSMA3 did not match any columns in phosphoproteomics dataframe. PSMA3_phosphoproteomics column inserted, but filled with NaN.
RARRES2 did not match any columns in phosphoproteomics dataframe. RARRES2_phosphoproteomics column inserted, but filled with NaN.
VAV3 did not match any columns in phosphoproteomics dataframe. VAV3_phosphoproteomics column inserted, but filled with NaN.
PES1 did not match any columns in phosphoproteomics dataframe. PES1_phosphoproteomics column inserted, but filled with NaN.
MTHFD1 did not match any columns in phosphoproteomics dataframe. MTHFD1_phosphoproteomics column inserted, but filled with NaN.
GTPBP10 did not match any columns in phosphoproteomics dataframe. GTPBP10_phosphoproteomics column inserted, but fil

C1orf109 did not match any columns in phosphoproteomics dataframe. C1orf109_phosphoproteomics column inserted, but filled with NaN.
NOTCH1 did not match any columns in phosphoproteomics dataframe. NOTCH1_phosphoproteomics column inserted, but filled with NaN.
ABCB10 did not match any columns in phosphoproteomics dataframe. ABCB10_phosphoproteomics column inserted, but filled with NaN.
SLC25A23 did not match any columns in phosphoproteomics dataframe. SLC25A23_phosphoproteomics column inserted, but filled with NaN.
ELOB did not match any columns in phosphoproteomics dataframe. ELOB_phosphoproteomics column inserted, but filled with NaN.
TSPAN13 did not match any columns in phosphoproteomics dataframe. TSPAN13_phosphoproteomics column inserted, but filled with NaN.
SFRP4 did not match any columns in phosphoproteomics dataframe. SFRP4_phosphoproteomics column inserted, but filled with NaN.
TPP1 did not match any columns in phosphoproteomics dataframe. TPP1_phosphoproteomics column inserte

...

LY6E did not match any columns in phosphoproteomics dataframe. LY6E_phosphoproteomics column inserted, but filled with NaN.
SPTLC3 did not match any columns in phosphoproteomics dataframe. SPTLC3_phosphoproteomics column inserted, but filled with NaN.
OARD1 did not match any columns in phosphoproteomics dataframe. OARD1_phosphoproteomics column inserted, but filled with NaN.
PIBF1 did not match any columns in phosphoproteomics dataframe. PIBF1_phosphoproteomics column inserted, but filled with NaN.
PTAR1 did not match any columns in phosphoproteomics dataframe. PTAR1_phosphoproteomics column inserted, but filled with NaN.
LYRM9 did not match any columns in phosphoproteomics dataframe. LYRM9_phosphoproteomics column inserted, but filled with NaN.
HAPLN1 did not match any columns in phosphoproteomics dataframe. HAPLN1_phosphoproteomics column inserted, but filled with NaN.
EXT1 did not match any columns in phosphoproteomics dataframe. EXT1_phosphoproteomics column inserted, but filled wi

MITD1 did not match any columns in phosphoproteomics dataframe. MITD1_phosphoproteomics column inserted, but filled with NaN.
NUDT4 did not match any columns in phosphoproteomics dataframe. NUDT4_phosphoproteomics column inserted, but filled with NaN.
MICU1 did not match any columns in phosphoproteomics dataframe. MICU1_phosphoproteomics column inserted, but filled with NaN.
IFT22 did not match any columns in phosphoproteomics dataframe. IFT22_phosphoproteomics column inserted, but filled with NaN.
TXNL4B did not match any columns in phosphoproteomics dataframe. TXNL4B_phosphoproteomics column inserted, but filled with NaN.
HAUS7 did not match any columns in phosphoproteomics dataframe. HAUS7_phosphoproteomics column inserted, but filled with NaN.
TFF2 did not match any columns in phosphoproteomics dataframe. TFF2_phosphoproteomics column inserted, but filled with NaN.
NDUFB1 did not match any columns in phosphoproteomics dataframe. NDUFB1_phosphoproteomics column inserted, but filled 

ANXA8L1 did not match any columns in phosphoproteomics dataframe. ANXA8L1_phosphoproteomics column inserted, but filled with NaN.
SERPINB6 did not match any columns in phosphoproteomics dataframe. SERPINB6_phosphoproteomics column inserted, but filled with NaN.
PODNL1 did not match any columns in phosphoproteomics dataframe. PODNL1_phosphoproteomics column inserted, but filled with NaN.
PIAS3 did not match any columns in phosphoproteomics dataframe. PIAS3_phosphoproteomics column inserted, but filled with NaN.
CNTNAP1 did not match any columns in phosphoproteomics dataframe. CNTNAP1_phosphoproteomics column inserted, but filled with NaN.
PI16 did not match any columns in phosphoproteomics dataframe. PI16_phosphoproteomics column inserted, but filled with NaN.
NDUFS5 did not match any columns in phosphoproteomics dataframe. NDUFS5_phosphoproteomics column inserted, but filled with NaN.
SDHAF1 did not match any columns in phosphoproteomics dataframe. SDHAF1_phosphoproteomics column inser

GSDMA did not match any columns in phosphoproteomics dataframe. GSDMA_phosphoproteomics column inserted, but filled with NaN.
ATIC did not match any columns in phosphoproteomics dataframe. ATIC_phosphoproteomics column inserted, but filled with NaN.
NPRL3 did not match any columns in phosphoproteomics dataframe. NPRL3_phosphoproteomics column inserted, but filled with NaN.
HLCS did not match any columns in phosphoproteomics dataframe. HLCS_phosphoproteomics column inserted, but filled with NaN.
MRPL20 did not match any columns in phosphoproteomics dataframe. MRPL20_phosphoproteomics column inserted, but filled with NaN.
CELF2 did not match any columns in phosphoproteomics dataframe. CELF2_phosphoproteomics column inserted, but filled with NaN.
MT1X did not match any columns in phosphoproteomics dataframe. MT1X_phosphoproteomics column inserted, but filled with NaN.
C8orf44-SGK3 did not match any columns in phosphoproteomics dataframe. C8orf44-SGK3_phosphoproteomics column inserted, but

MPV17L2 did not match any columns in phosphoproteomics dataframe. MPV17L2_phosphoproteomics column inserted, but filled with NaN.
FHL5 did not match any columns in phosphoproteomics dataframe. FHL5_phosphoproteomics column inserted, but filled with NaN.
RPL35A did not match any columns in phosphoproteomics dataframe. RPL35A_phosphoproteomics column inserted, but filled with NaN.
SV2A did not match any columns in phosphoproteomics dataframe. SV2A_phosphoproteomics column inserted, but filled with NaN.
BCAP29 did not match any columns in phosphoproteomics dataframe. BCAP29_phosphoproteomics column inserted, but filled with NaN.
SERPING1 did not match any columns in phosphoproteomics dataframe. SERPING1_phosphoproteomics column inserted, but filled with NaN.
TRAPPC6A did not match any columns in phosphoproteomics dataframe. TRAPPC6A_phosphoproteomics column inserted, but filled with NaN.
FAM49B did not match any columns in phosphoproteomics dataframe. FAM49B_phosphoproteomics column inser

## Conclusions

TODO